In [1]:
import spotipy
import csv
import os
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from dotenv import load_dotenv

In [2]:
load_dotenv()

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

client_credentials_manager = SpotifyClientCredentials()
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

In [3]:
search_result = spotify.search(q='2022', limit=50) #the result only consists of tracks

tracks = search_result['tracks']['items']

for i in range(19):
    search_result = spotify.next(search_result['tracks'])
    tracks.extend(search_result['tracks']['items'])

In [10]:
tracks[0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [4]:
artists_id = []
for i in range(len(tracks)):
    for j in range(len(tracks[i]['artists'])):
        artists_id.append(tracks[i]['artists'][j]['id'])

artists_id = list(dict.fromkeys(artists_id))

In [6]:
artists = []
for i in range(len(artists_id)):
    artist_info = spotify.artist(artist_id=artists_id[i])
    cols = ['id', 'name', 'genres', 'followers', 'popularity']
    artist = dict.fromkeys(cols)
    for key in artist:
        if key != 'followers' and key != 'genres':
            artist[key] = artist_info[key]
        elif key != 'genres':
            artist[key] = artist_info[key]['total']
        else:
            artist[key] = ', '.join(artist_info[key])
    artists.append(artist)

In [11]:
with open('artists.tsv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=artists[0].keys(), delimiter='\t')
    writer.writeheader()
    for data in artists:
        writer.writerow(data)